# Reading, writing a MED file using the basic MEDLoader API

The basic MEDLoader API is contained in the `MEDLoader` class. All methods of this class are static (they do not depend on a particular instance of the class), their names begin with a capital letter. All read/write operations are executed with each method call, and no internal state of the class is saved.

## Objective

Write a mesh and a field from scratch, reread them, and compare the results.

Topics covered: using the basic `MEDLoader` API

> - Writing a file
> - Reading a file

## Implementation Start

We import the `MEDCoupling` python package named `medcoupling`. The functions of `MEDLoader` are included in `medcoupling`.

In [ ]:
import medcoupling as mc

## Mesh Reading, Writing

First, let's create a mesh `targetMesh` composed of several geometric types.

The coordinate array of the mesh is set from a `DataArray`:

In [ ]:
targetMesh = mc.MEDCouplingUMesh("MyMesh", 2)

# fmt: off
targetCoords = [
    -0.3, -0.3, 0.2, -0.3, 0.7, -0.3, -0.3, 0.2, 0.2,
     0.2,  0.7, 0.2, -0.3, 0.7,  0.2,  0.7, 0.7, 0.7,
]
# fmt: on

myCoords = mc.DataArrayDouble(targetCoords, 9, 2)
myCoords.setInfoOnComponents(mc.svec(["X [km]", "Y [mm]"]))
targetMesh.setCoords(myCoords)

Cells are inserted in the mesh. Cells are defined as a set of index of points (tuple) from the
`DataArray` of coordinates of the mesh.

For instance, `tri1 = [1, 4, 2]` designate a triangle cell composed of the
first, fourth and second points of the coordinate array.

In [ ]:
tri1 = [1, 4, 2]
tri2 = [4, 5, 2]
qua1 = [0, 3, 4, 1]
qua2 = [6, 7, 4, 3]
qua3 = [7, 8, 5, 4]

targetMesh.allocateCells(5)  # allocate memory
targetMesh.insertNextCell(mc.NORM_TRI3, 3, tri1)
targetMesh.insertNextCell(mc.NORM_TRI3, 3, tri2)
targetMesh.insertNextCell(mc.NORM_QUAD4, 4, qua1)
targetMesh.insertNextCell(mc.NORM_QUAD4, 4, qua2)
targetMesh.insertNextCell(mc.NORM_QUAD4, 4, qua3)

<div class="alert alert-block alert-success">
<b>Note:</b> The mesh targetMesh is ordered by geometric type.</div>

The mesh can then be directly written...

In [ ]:
mc.WriteUMesh("TargetMesh.med", targetMesh, True)  # True means 'from scratch'

... and read.

In [ ]:
meshRead: mc.MEDCouplingUMesh
meshRead = mc.ReadUMeshFromFile("TargetMesh.med", targetMesh.getName(), 0)
print("Is the read mesh equal to 'targetMesh' ?", meshRead.isEqual(targetMesh, 1e-12))

## Read/Write a Field on a Time Step

We now create a vector field `f` on cells (P0) with `targetMesh` as its support. This field corresponds, for example, to the physical time 5.6, marked by iteration 7 and sub-iteration 8. We take this opportunity to remind you that in `MEDCoupling` fields, physical time is given for information only; storage and most API functions are based on the last two integers.

In [ ]:
f: mc.MEDCouplingFieldDouble
f = mc.MEDCouplingFieldDouble.New(mc.ON_CELLS, mc.ONE_TIME)
f.setTime(5.6, 7, 8)  # Declare the timestep associated with the field
f.setArray(targetMesh.computeCellCenterOfMass())
f.setMesh(targetMesh)
f.setName("AFieldName")
mc.WriteField("MyFirstField.med", f, True)

Subsidiary question: What does the field created correspond to?

<div class="alert alert-block alert-success">
<b>Note:</b> The mesh and the field are written in one go to the file "MyFirstField.med".</div>

We then read MyFirstField.med:

In [ ]:
f2: mc.MEDCouplingFieldDouble
f2 = mc.ReadFieldCell("MyFirstField.med", f.getMesh().getName(), 0, f.getName(), 7, 8)
print("Is the read field identical to 'f' ?", f2.isEqual(f, 1e-12, 1e-12))

<div class="alert alert-block alert-success">
<b>Note:</b> When reading the field, we must know its name, the name of its support mesh, and the desired timestep. Functions like MEDFileFields.getFieldsNames() or MEDFileMeshes.getMeshesNames() help with this.</div>

<div class="alert alert-block alert-success">
<b>Note:</b> The name ReadFieldCell() reminds us that the field must be read on the cells. Remember that according to the MED file standard, the same field can have some of its data stored on cells, but also simultaneously on nodes, Gauss points, etc... even if this kind of exotic mix is generally not recommended.</div>

## Read/Write a Field on Multiple Time Steps

Here, unlike the previous case, we write multiple times to the same MED file. First, let's write the mesh.

In [ ]:
mc.WriteUMesh("MySecondField.med", f.getMesh(), True)

Then, we write only the information related to the field (primarily its values array).

In [ ]:
mc.WriteFieldUsingAlreadyWrittenMesh("MySecondField.med", f)  # mesh is not re-written

Next, we add a second timestep on the same mesh.

In [ ]:
f2: mc.MEDCouplingFieldDouble
f2 = f.clone(True)  # 'True' means that we need a deep copy
f2.getArray()[:] = 2.0
f2.setTime(7.8, 9, 10)
mc.WriteFieldUsingAlreadyWrittenMesh("MySecondField.med", f2)

Now the file "MySecondField.med" contains the mesh and a field with two timesteps carried by this mesh.

We can reread all of this with methods similar to what was seen previously:

In [ ]:
f3: mc.MEDCouplingFieldDouble
f3 = mc.ReadFieldCell("MySecondField.med", f.getMesh().getName(), 0, f.getName(), 7, 8)
print("Is the field read in the file equal to 'f' ?", f.isEqual(f3, 1e-12, 1e-12))
f4: mc.MEDCouplingFieldDouble
f4 = mc.ReadFieldCell("MySecondField.med", f.getMesh().getName(), 0, f.getName(), 9, 10)
print("Is the field read in the file equal to 'f2' ?", f2.isEqual(f4, 1e-12, 1e-12))